In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch

hf_token = ''
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.1-8B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
    token=hf_token,
)

![](https://i.imgur.com/Y8qsNvf.png)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

<a name="Data"></a>
### Data Prep
**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!
![](https://i.imgur.com/cIGv8Cb.png)
![](https://i.imgur.com/FFxWTbK.png)
![](https://i.imgur.com/a3navcZ.png)

In [ ]:
from datasets import load_dataset

# Define prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}"""

# Define EOS token (ensure tokenizer has this token)
EOS_TOKEN = tokenizer.eos_token if tokenizer.eos_token else "<|endoftext|>"


# Formatting function for dataset
def formatting_prompts_func(examples):
    texts = [
        alpaca_prompt.format(
            instruction=instruction, input=input_text, output=output
        )
        + EOS_TOKEN
        for instruction, input_text, output in zip(
            examples["instruction"], examples["input"], examples["output"]
        )
    ]
    return {"text": texts}


# Load and process dataset
dataset = load_dataset("ThanhTranVisma/blueVi-GPT-dataset", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

![](https://i.imgur.com/D8sDuhK.png)
![](https://i.imgur.com/YeGVUup.png)



In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

In [ ]:
trainer_stats = trainer.train()

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}"""

FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction="",  # instruction
            input="What is your name? Who created you?",  # input
            output="",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")


from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

In [ ]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction="Anonymize the data",  # instruction
            input="""John Doe's email is J.Simpson@netwrix.com. His BSN is 123456789.
        His home address is 10 Langelo. His ZIP code is 7666MC.
        His MasterCard number is 5258704108753590 and his Visa number is 4563-7568-5698-4587.
        His IBAN number is NL91ABNA0417164300.
        His date of birth is 01/01/1990. His IP address is 192.168.1.1.

        Olivia Doe's email is D.Olivia@netwrix.com. Her BSN is 987654321.
        Her home address is 60 Langelo, Langelo. Her ZIP code is 7612MC.
        Her MasterCard number is 5258704108753577 and her Visa number is 4511-7500-0000-0000.
        Her IBAN number is DE91ABNA0417164300.
        Her date of birth is 01/01/1991. Her IP address is 192.168.2.2.

        I saw John and Olivia at the park. They were having a great time.""",  # input
            output="",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

In [ ]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction="Determine and return the most suitable instruction: either 'Default' or 'Operation instruction",  # instruction
            input="Please provide the details to create a new operation.",  # input
            output="",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

In [ ]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction="Determine if the provided data contains personal information. Return True if it does and False otherwise. Note: The data might include inaccuracies, so evaluate carefully.",  # instruction
            input="llice Smith's email is allice.smith@example..com. Her BSN is 12345678. Her address is 15 Elm Street, and her zip code is 12345.",  # input
            output="",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

In [ ]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction="Determine if the provided data contains personal information. Return True if it does and False otherwise. Note: The data might include inaccuracies, so evaluate carefully.",  # instruction
            input="Hi, what a nice day today.",  # input
            output="",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

In [ ]:
# Enable native 2x faster inference
model = FastLanguageModel.for_inference(model)

# Define the Alpaca-style prompt template
alpaca_prompt = (
    "Extract operation details from the input, structure them in the following JSON format, and return the result:\n"
    "{\n"
    "  \"name\": \"\",\n"
    "  \"description\": \"\",\n"
    "  \"duration\": 0,\n"
    "  \"forAppointment\": true,\n"
    "  \"vatRate\": 0,\n"
    "  \"invoicing\": false,\n"
    "  \"hourlyRate\": 0,\n"
    "  \"unitPrice\": null,\n"
    "  \"operationRateType\": null,\n"
    "  \"methodsOfConsult\": [\n"
    "    \"TEL\",\n"
    "    \"WEB\",\n"
    "    \"LOC\"\n"
    "  ],\n"
    "  \"wizard\": null\n"
    "}"
)

# Construct the input prompt
input_prompt = (
    alpaca_prompt
    + "\nCreate a new operation called 'Webinar' with no invoicing applied, unit price 50."
)

# Tokenize the input
inputs = tokenizer(
    [input_prompt],
    return_tensors="pt",
).to("cuda")

# Import TextStreamer for streaming output
from transformers import TextStreamer

# Initialize the text streamer
text_streamer = TextStreamer(tokenizer)

# Generate the response using the model
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

In [ ]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction="",  # instruction
            input="Who created you?",  # input
            output="",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# Save the model in the merged format
model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")
# Push the merged model to the Hugging Face Hub
model.push_to_hub_merged(
    "ThanhTranVisma/Llama-3.1-8B-blueVi",
    tokenizer=tokenizer,
    save_method="merged_16bit",
    token=hf_token,
)

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Push the model with different quantization methods to the Hub
quant_methods = ["q4_k_m", "q8_0"]
for quant in quant_methods:
    model.push_to_hub_gguf(
        "ThanhTranVisma/Llama-3.1-8B-blueVi",
        tokenizer=tokenizer,
        quantization_method=quant_methods,
        token=hf_token,
    )

## Next steps

![](https://i.imgur.com/dMLEDKH.png)
![](https://i.imgur.com/jaOowAJ.png)
![](https://i.imgur.com/DlTKPHj.png)
![](https://i.imgur.com/EMBelvN.png)
